In [1]:
# the scipy version packaged with colab is not tolerant of misformated WAV files.
# install the latest version.
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

fatal: destination path 'tortoise-tts' already exists and is not an empty directory.
/content/tortoise-tts
  Using cached numba-0.48.0.tar.gz (2.0 MB)
  Preparing metadata (setup.py) ... done
  Using cached ffmpeg-1.4-py3-none-any.whl
  Using cached llvmlite-0.31.0.tar.gz (110 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. This could take a while.
  Using cached librosa-0.10.0.post2-py3-none-any.whl (253 kB)
  Using cached librosa-0.10.0.post1-py3-none-any.whl (252 kB)
  Using cached librosa-0.10.0-py3-none-any.whl (252 kB)
  Using cached librosa-0.9.2-py3-none-any.whl (214 kB)
  Using cached resampy-0.4.2-py3-none-any.whl (3.1 MB)
INFO: pip is looking at multiple versions of resampy to determine which version is compatible with other requirements. This could take a while.
  Using cached resampy-0.4.1-py3-none-any.whl (3.1 MB)
  Using cached resampy-0.4.0-py3-none-any.wh

In [2]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

In [3]:
# This is the text that will be spoken.
text = "I had no dream no future nothing I eat one for at least two months I work or jobs I did and I am today so I am saying if I was able to do it you can the honour is really mine for me to be standing here among all of the you on this very special day of yours not many I was there among all of you and I wish that all of you become more famous more rickshaw powerful than me I sincerely wish that for everyone because it is possible when I was studying my mum and dad told me study study study study study study study study study study study study study study"
# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "fast"

In [6]:
# Optionally, upload use your own voice by running the next two cells. I recommend
# you upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.
CUSTOM_VOICE_NAME = "prasadv"

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

Saving voice_0_0.wav to voice_0_0 (1).wav
Saving voice_0_1.wav to voice_0_1 (1).wav
Saving voice_0_2.wav to voice_0_2 (1).wav
Saving voice_0_3.wav to voice_0_3 (1).wav
Saving voice_0_4.wav to voice_0_4 (1).wav
Saving voice_0_5.wav to voice_0_5 (1).wav
Saving voice_0_6.wav to voice_0_6 (1).wav
Saving voice_0_7.wav to voice_0_7 (1).wav
Saving voice_0_8.wav to voice_0_8 (1).wav
Saving voice_0_9.wav to voice_0_9 (1).wav
Saving voice_0_10.wav to voice_0_10 (1).wav
Saving voice_0_11.wav to voice_0_11 (1).wav
Saving voice_0_12.wav to voice_0_12 (1).wav
Saving voice_0_13.wav to voice_0_13 (1).wav
Saving voice_0_14.wav to voice_0_14 (1).wav
Saving voice_0_15.wav to voice_0_15 (1).wav
Saving voice_0_16.wav to voice_0_16 (1).wav
Saving voice_0_17.wav to voice_0_17 (1).wav
Saving voice_0_18.wav to voice_0_18 (1).wav
Saving voice_0_19.wav to voice_0_19 (1).wav


In [12]:
pip install translate

In [7]:
from translate import Translator

# Define the input text in English, Telugu, and Hindi
texts = {
    "english": "I had no dream no future nothing. I eat one for at least two months. I work two jobs I did, and I am here today. So I am saying if I was able to do it, you can do it too.",
    "telugu": "నాకు ఎవరు సపన లేదు, భవిష్యత్తు లేదు. కనీసం రెండు నెలల కురిసి ఒకటి తిన్నాను. రెండు ఉద్యోగాలు చేసాను, మరియు నాకు ఇక్కడ ఉన్నాను. ఇంకా, అదేమి చేయగలనుకుంటున్నాను. కాబట్టి, నాకు ఇక్కడ ఉన్నట్లు మీరు చేయగలరు.",
    "hindi": "मेरे पास कोई सपना नहीं था, कोई भविष्य नहीं था। मैंने कम से कम दो महीने तक एक को खाया। मैंने दो नौकरियाँ की थी और आज मैं यहाँ हूँ। इसलिए मैं कह रहा हूँ कि अगर मैं कर सकता हूँ, तो आप भी कर सकते हैं।"
}

def translate_text(input_text, target_lang):
    translator = Translator(to_lang=target_lang)
    translation = translator.translate(input_text)
    return translation

# Translate the English text to Telugu and Hindi
translated_texts = {
    "english": texts['english'],
    "telugu": translate_text(texts["english"], "te"),
    "hindi": translate_text(texts["english"], "hi")
}

# Print the translated texts
for lang, text in translated_texts.items():
    print(f"{lang.capitalize()}: {text}")


English: I had no dream no future nothing. I eat one for at least two months. I work two jobs I did, and I am here today. So I am saying if I was able to do it, you can do it too.
Telugu: నాకు కలలు లేవు, భవిష్యత్తు ఏమీ లేదు. నేను కనీసం రెండు నెలలు ఒకటి తింటాను. నేను చేసిన రెండు ఉద్యోగాలను నేను పని చేస్తున్నాను మరియు నేను ఈ రోజు ఇక్కడ ఉన్నాను. కాబట్టి నేను చెప్తున్నాను, నేను దానిని చేయగలిగితే, మీరు కూడా చేయగలరు.
Hindi: मेरा कोई सपना नहीं था, कोई भविष्य नहीं था। मैं कम से कम दो महीने के लिए एक खाता हूं। मैंने दो काम किए हैं, और मैं आज यहाँ हूँ। इसलिए मैं कह रहा हूं कि अगर मैं ऐसा करने में सक्षम था, तो आप भी ऐसा कर सकते हैं।


In [ ]:
# Generate speech with the custom voice for English, Telugu, and Hindi
import IPython.display as ipd

for lang, text in translated_texts.items():
    voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
    gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                              preset="fast")
    output_file = f'generated-{CUSTOM_VOICE_NAME}-{lang}.wav'
    torchaudio.save(output_file, gen.squeeze(0).cpu(), 24000)
    display(ipd.Audio(output_file))

Generating autoregressive samples..


100%|██████████| 6/6 [01:07<00:00, 11.21s/it]


Computing best candidates using CLVP and CVVP


100%|██████████| 6/6 [00:28<00:00,  4.83s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:43<00:00,  1.86it/s]
